In [ ]:
import glob
import os
import pandas as pd

In [ ]:
patcmd = '/usr/bin/java -Xmx2024m -jar ./LIUM_SpkDiarization-8.4.1.jar --fInputMask="%s" --sOutputMask="%s" --doCEClustering "%s"'
wavdir = '../../wav/' #should be updated after wav download from FTP

for f in glob.glob(wavdir + '*.wav'):
    base, _ = os.path.splitext(os.path.basename(f))
    out = '../../automatic_baselines/liumspkdirization_raw/%s.seg' % base
    if os.path.exists(out):
        continue
    cmd = patcmd % (f, out, base)
    print(cmd)
    os.system(cmd)

In [ ]:
names = ['show', 'skip', 'startcs', 'durcs', 'gender', 'canal', 'uni', 'spkr']
for f in glob.glob('../../automatic_baselines/liumspkdirization_raw/*.seg'):
    print(f)
    base, _ = os.path.splitext(os.path.basename(f))
    df = pd.read_csv(f, sep=' ', comment=';', header=None, names=names)
    df = df.sort_values(by=['startcs'])
    retdf = pd.DataFrame()
    retdf['start'] = df.startcs / 100.
    retdf['stop'] = (df.startcs + df.durcs) / 100.
    retdf['label'] = df.gender.map(lambda x : 'female' if x == 'F' else 'male')
    retdf['canal'] = df.canal.map(lambda x: 'telephone' if x == 'T' else 'studio')
    retdf.to_csv('../../automatic_baselines/liumspkdirization_csv/%s.csv' % base, index = False)